
Step 1. [Download history price and clean data](/notebooks/forex/hawkeye/1_download.ipynb)

Step 2. [Use monte carlo simulate each stock's movement for 10000 times](/notebooks/forex/hawkeye/2_process.ipynb)

**Step 3. Apply some heuristic rules to give stock recommendation**

In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('/opt/hawkeye')
from asx import *

path = '/opt/hawkeye/data/20190524/result.csv'
last_path = '/opt/hawkeye/data/20190517/result.csv'

df = pd.read_csv(path,
                 usecols=[
                     'code', 'last_date', 'start price', 'sim_mean',
                     'sim_diff', 'VaR 99%', 'VaR 99% Percent', 'volume_mean',
                     'return_mean', 'return_sigma'
                 ],
                 index_col='code')
last_df = pd.read_csv(last_path,
                      usecols=[
                          'code', 'last_date', 'start price', 'sim_mean',
                          'sim_diff', 'VaR 99%', 'VaR 99% Percent',
                          'volume_mean', 'return_mean', 'return_sigma'
                      ],
                      index_col='code')
df.head()
df['return'] = round(df['sim_diff'] / df['start price'] * 100, 3)
df['last_return'] = round(last_df['sim_diff'] / last_df['start price'] * 100,
                          3)
df['return_increase'] = df['return'] - df['last_return']
df['week_return'] = round(
    (df['start price'] - last_df['start price']) / df['start price'] * 100, 3)
df.drop(columns=['sim_diff', 'sim_mean', 'VaR 99%'], inplace=True)

print(f'Total stock number = {len(df)}')

print(df['start price'].quantile(0.7))
print('Sim return 90%% percentile = %s' % df['return'].quantile(0.9))

volume_threshold = df['volume_mean'].quantile(0.1)
print(f'Volume 90% percentile = {volume_threshold}')

df = df[(df['volume_mean'] > volume_threshold) & (df['start price'] > 0.1)]
print(len(df))
df.sort_values(by='return', ascending=False).head(10)

Total stock number = 940
1.5654999999999983
Sim return 90% percentile = 1.1709
Volume 90% percentile = 70810.93838383838
602


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
ANO,2019-05-27,7.020,10.5369,1.023357e+05,0.022919,0.066730,4.705,3.639,1.066,33.048
AVH,2019-05-27,0.455,12.6562,8.791681e+06,0.020646,0.075820,3.976,4.350,-0.374,-7.692
LTR,2019-05-28,0.110,15.9038,1.165828e+07,0.018294,0.092349,3.816,2.795,1.021,35.455
8CO,2019-05-27,0.155,13.3160,2.528027e+05,0.017755,0.075269,3.636,3.297,0.339,0.000
WZR,2019-05-28,0.170,17.7403,5.523038e+06,0.018665,0.100016,3.627,3.359,0.268,17.647
ATU,2019-05-27,0.340,13.0927,7.034715e+05,0.016598,0.075959,3.341,3.331,0.010,-2.941
ISX,2019-05-27,0.660,9.4187,2.020696e+06,0.016576,0.056584,3.180,1.653,1.527,49.242
LBT,2019-05-28,0.205,28.4314,9.562466e+05,0.015325,0.151159,3.003,0.011,2.992,60.000
WKT,2019-05-28,0.315,8.9623,7.337652e+05,0.013642,0.052351,2.707,2.563,0.144,11.111


In [47]:
# high mean return 
df.sort_values(by='return', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
ANO,2019-05-27,7.020,10.5369,1.023357e+05,0.022919,0.066730,4.705,3.639,1.066,33.048
AVH,2019-05-27,0.455,12.6562,8.791681e+06,0.020646,0.075820,3.976,4.350,-0.374,-7.692
LTR,2019-05-28,0.110,15.9038,1.165828e+07,0.018294,0.092349,3.816,2.795,1.021,35.455
8CO,2019-05-27,0.155,13.3160,2.528027e+05,0.017755,0.075269,3.636,3.297,0.339,0.000
WZR,2019-05-28,0.170,17.7403,5.523038e+06,0.018665,0.100016,3.627,3.359,0.268,17.647
ATU,2019-05-27,0.340,13.0927,7.034715e+05,0.016598,0.075959,3.341,3.331,0.010,-2.941
ISX,2019-05-27,0.660,9.4187,2.020696e+06,0.016576,0.056584,3.180,1.653,1.527,49.242
LBT,2019-05-28,0.205,28.4314,9.562466e+05,0.015325,0.151159,3.003,0.011,2.992,60.000
WKT,2019-05-28,0.315,8.9623,7.337652e+05,0.013642,0.052351,2.707,2.563,0.144,11.111


In [48]:
# high price
df[df['start price']>1].sort_values(by='return', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
ANO,2019-05-27,7.020,10.5369,1.023357e+05,0.022919,0.066730,4.705,3.639,1.066,33.048
5GN,2019-05-27,1.350,8.0450,3.169550e+05,0.012917,0.046811,2.579,2.371,0.208,9.630
Z1P,2019-05-28,3.460,7.2543,2.413338e+06,0.012759,0.042866,2.412,2.547,-0.135,-1.734
BUB,2019-05-27,1.265,9.5571,5.385370e+06,0.011732,0.053853,2.362,2.220,0.142,0.791
CIA,2019-05-27,2.900,6.7832,2.541779e+05,0.010158,0.039037,1.979,2.007,-0.028,3.103
JIN,2019-05-27,17.130,4.5346,3.068755e+05,0.009134,0.027798,1.802,1.883,-0.081,-2.977
MGX,2019-05-27,1.245,4.5381,3.009814e+06,0.009362,0.028459,1.773,1.950,-0.177,-0.803
NEA,2019-05-27,3.440,5.8477,2.918665e+06,0.008758,0.033279,1.752,1.991,-0.239,-11.337
CRD,2019-05-27,1.660,8.1398,2.656470e+05,0.008834,0.043928,1.750,1.861,-0.111,-3.916


In [49]:
# highest price
df[df['return']>0.8].sort_values(by='start price', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
MFG,2019-05-27,43.90,2.9261,5.647130e+05,0.006793,0.019041,1.330,1.392,-0.062,-2.096
CUV,2019-05-27,33.68,6.2294,1.138231e+05,0.006901,0.033153,1.339,1.211,0.128,6.591
APX,2019-05-27,26.24,5.6605,8.151889e+05,0.007777,0.032044,1.559,1.614,-0.055,4.002
APT,2019-05-27,23.93,6.8776,2.080330e+06,0.007714,0.038569,1.505,1.696,-0.191,-6.979
BRG,2019-05-27,17.28,4.2281,2.973136e+05,0.005489,0.023411,1.061,1.158,-0.097,-5.382
JIN,2019-05-27,17.13,4.5346,3.068755e+05,0.009134,0.027798,1.802,1.883,-0.081,-2.977
IEL,2019-05-27,16.59,4.7205,7.116587e+05,0.005701,0.025645,1.111,1.163,-0.052,-0.784
IVC,2019-05-27,15.97,3.6336,5.125682e+05,0.004838,0.020115,0.948,0.884,0.064,3.193
WEB,2019-05-28,15.15,7.3906,5.650480e+05,0.004333,0.037057,0.831,0.951,-0.120,-10.891


In [50]:
# low risk
var_threshold = df['VaR 99% Percent'].quantile(0.3)
print(var_threshold)
df[df['VaR 99% Percent'] < var_threshold].sort_values(by='return',
                                                      ascending=False).head(10)

4.15307


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
AD8,2019-05-27,7.120,4.0520,1.861000e+05,0.007307,0.024062,1.440,1.438,0.002,0.140
MFG,2019-05-27,43.900,2.9261,5.647130e+05,0.006793,0.019041,1.330,1.392,-0.062,-2.096
DDR,2019-05-27,5.120,3.4662,1.442235e+05,0.006555,0.021161,1.264,1.281,-0.017,-0.391
GOR,2019-05-27,1.090,3.9340,2.647559e+06,0.005240,0.021955,1.001,1.043,-0.042,7.339
IVC,2019-05-27,15.970,3.6336,5.125682e+05,0.004838,0.020115,0.948,0.884,0.064,3.193
ASB,2019-05-27,2.860,3.3153,9.636269e+05,0.004389,0.018662,0.841,0.726,0.115,4.196
CSR,2019-05-27,4.090,3.3156,3.586410e+06,0.004111,0.018461,0.816,0.446,0.370,15.648
SUL,2019-05-28,9.295,3.7704,1.093667e+06,0.004167,0.019922,0.806,0.136,0.670,18.827
XRO,2019-05-28,61.730,4.0670,4.437772e+05,0.004101,0.021583,0.787,0.936,-0.149,0.697


In [59]:
# high volume
df.sort_values(by='volume_mean', ascending=False).head(20)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
TLS,2019-05-28,3.540,2.2893,2.656801e+07,0.002428,0.012408,0.476,0.439,0.037,0.847
FMG,2019-05-27,8.450,4.6365,1.957003e+07,0.008591,0.027545,1.673,1.646,0.027,-5.917
S32,2019-05-28,3.475,2.9987,1.836221e+07,0.000969,0.014498,0.215,0.293,-0.078,-1.295
HSO,2019-05-27,2.460,1.3373,1.400865e+07,0.001342,0.006879,0.253,0.380,-0.127,0.407
MGR,2019-05-27,3.120,1.6390,1.283096e+07,0.003588,0.010326,0.690,0.510,0.180,5.449
AWC,2019-05-27,2.430,3.6027,1.219895e+07,0.001813,0.017312,0.364,0.334,0.030,1.646
VCX,2019-05-28,2.700,2.2497,1.218716e+07,0.000752,0.010502,0.156,-0.044,0.200,4.444
SCG,2019-05-27,3.920,2.2878,1.188383e+07,0.000541,0.010721,0.084,-0.034,0.118,2.041
AMP,2019-05-27,2.180,5.2341,1.176366e+07,-0.000726,0.021988,-0.148,-0.064,-0.084,-0.917


In [62]:
# highest market cap.
highest_market_cap = [
    'BHP', 'RIO', 'CBA', 'WBC', 'CSL', 'ANZ', 'NAB', 'MQG', 'WOW', 'WES',
    'TLS', 'WPL'
    'S32', 'FMG', 'BXB', 'NCM', 'IAG', 'SUN', 'AMC', 'QBE', 'CIM', 'ALL',
    'AGL', 'ASX', 'RHC', 'OSH', 'SHL', 'TWE', 'REA', 'COH'
]
asx_20 = get_asx_20_list()
df[df.index.isin(asx_20)].sort_values(by='return', ascending=False).head(20)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
RIO,2019-05-27,103.020,2.4301,1.769566e+06,0.003350,0.013688,0.648,0.647,0.001,1.621
TLS,2019-05-28,3.540,2.2893,2.656801e+07,0.002428,0.012408,0.476,0.439,0.037,0.847
AMC,2019-05-27,16.160,1.3647,3.744241e+06,0.002333,0.007838,0.454,0.532,-0.078,-2.723
BXB,2019-05-27,12.190,1.7681,3.730137e+06,0.002195,0.009808,0.444,0.453,-0.009,-3.035
WES,2019-05-28,38.150,2.3952,2.461372e+06,0.002222,0.012288,0.430,0.335,0.095,4.692
ANZ,2019-05-27,27.820,3.0649,5.665766e+06,0.001955,0.014615,0.394,0.242,0.152,7.081
WPL,2019-05-28,36.005,2.2743,2.371163e+06,0.001754,0.011270,0.350,0.520,-0.170,-3.319
WBC,2019-05-28,27.920,3.1289,6.702718e+06,0.001572,0.015265,0.319,0.195,0.124,8.990
IAG,2019-05-27,7.890,2.2053,5.293465e+06,0.001560,0.010892,0.301,0.342,-0.041,-1.774


## Thisk week recommendations

In [53]:
# 2019-05-17 recommendations
selection = [
    'AVH',  # high return & high risk
    'ANO',  # high return & high risk
    'AGH',  # high return & meduim risk
    'Z1P',  # medium price & risk
    'MFG',  # high price & low risk
    'AD8',  # low risk
    'FMG',  # high volume & low risk
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
ANO,2019-05-27,7.020,10.5369,1.023357e+05,0.022919,0.066730,4.705,3.639,1.066,33.048
AVH,2019-05-27,0.455,12.6562,8.791681e+06,0.020646,0.075820,3.976,4.350,-0.374,-7.692
Z1P,2019-05-28,3.460,7.2543,2.413338e+06,0.012759,0.042866,2.412,2.547,-0.135,-1.734
AGH,2019-05-27,0.660,8.2125,4.235954e+05,0.010286,0.045925,2.032,2.718,-0.686,6.061
FMG,2019-05-27,8.450,4.6365,1.957003e+07,0.008591,0.027545,1.673,1.646,0.027,-5.917
AD8,2019-05-27,7.120,4.0520,1.861000e+05,0.007307,0.024062,1.440,1.438,0.002,0.140
MFG,2019-05-27,43.900,2.9261,5.647130e+05,0.006793,0.019041,1.330,1.392,-0.062,-2.096


## Signle performance review

In [54]:
df[df.index=='FEX']

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
